#### 1、定义通用函数计时

In [1]:
import time
import math
import random
import pandas as pd

# 函数计时
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        stop_time = time.time()
        print('Func %s, run time:%s' % (func.__name__, stop_time - start_time))
        return res
    return wrapper

#### 2、加载数据集

In [2]:
class Dataset():
    def __init__(self, algo = 'UserCF'):
        self.project_path = './'
        self.train_path = 'Train_Test_Splits/Context/train.csv'
        self.test_path = 'Train_Test_Splits/Context/test.csv'
        self.train = {}
        self.test = {}
        if algo in ['UserCF', 'ItemCF', 'MostPopular', 'Random']:
            self.train, self.test = self.loadUserCFData()
    
    @timmer
    def loadUserCFData(self):
        train = pd.read_csv(self.project_path + self.train_path)
        test = pd.read_csv(self.project_path + self.test_path)
        train = train[['user_id', 'track_id']]
        test = test[['user_id', 'track_id']]
        return train, test
    
    @timmer
    def splitData(self, M, k, seed = 1):
        '''
        M: 划分的数目，最后取M折平均
        k: 本次是第k次划分
        '''
        self.train = self.train.sort_values(by=['user_id'])
        self.test = self.test.sort_values(by=['user_id'])
        
        train, test = [], []
        for index, row in self.train.iterrows():
            train.append((row['user_id'], row['track_id']))
        for index, row in self.test.iterrows():
            test.append((row['user_id'], row['track_id']))
        
        def convert_dict(data):
            data_dict = {}
            for user, item in data:
                if user not in data_dict:
                    data_dict[user] = set()
                data_dict[user].add(item)
            data_dict = {k: list(data_dict[k]) for k in data_dict}
            return data_dict  # {user1:(s1, s2, s3......)}
        
        return convert_dict(train), convert_dict(test)

#### 3、算法
- 随机算法Random
- 最热门MostPopular
- UserCF
- ItemCF

In [3]:
# 1.随机推荐
def Random(train, test, K, N):
    ''' 
    K，可忽略
    N，超参数，设置的TopN推荐物品数
    return: GetRecommendation, 推荐接口函数
    '''
    
    # 获取曲库中所有歌曲
    items = {}
    for user in train:
        for item in train[user]:
            items[item] = 1
    for user in test:
        for item in test[user]:
            items[item] = 1
    
    def GetRecommendation(user):
        #筛选掉用户已经评过分的物品
        user_items = set()
        if user in train:
            user_items = train[user]
        rec_items = {k: items[k] for k in items if k not in user_items} 
        
        #推荐不在该用户列表中的物品中的随机N个
        rec_items = list(rec_items.items()) 
        random.shuffle(rec_items)
        return rec_items[:N]
    
    return GetRecommendation


#最流行推荐
def MostPopular(train, test, K, N):
    ''' 
    K，可忽略
    N，超参数，设置的TopN推荐物品数
    return: GetRecommendation, 推荐接口函数
    '''
    items = {}
    for user in train:
        for item in train[user]:
            if item not in items:
                items[item] = 0
            items[item] += 1
    def GetRecommendation(user):
        user_items = set()
        if user in train:
            user_items = train[user]
            
        rec_items = {k:items[k] for k in items if k not in user_items}
        rec_items = list(sorted(rec_items.items(), key = lambda x:x[1], reverse=True)) #按物品点击量推荐前N项
        return rec_items[0:N]
    
    return GetRecommendation

# 3. 基于用户余弦相似度的推荐
def UserCF(train, test, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似用户数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算item->user的倒排索引
    item_users = {}
    for user in train:
        for item in train[user]:
            if item not in item_users:
                item_users[item] = set()
            item_users[item].add(user)
    
    # 计算用户相似度矩阵
    sim = {}
    num = {}
    for item in item_users:
        users = item_users[item]
        for u in users:
            if u not in num:
                num[u] = 0
            num[u] += 1
            if u not in sim:
                sim[u] = {}
            for v in users:
                if u == v: continue
                if v not in sim[u]:
                    sim[u][v] = 0
                sim[u][v] += 1
    for u in sim:
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u] * num[v])
    
    # 按照相似度排序
    sorted_user_sim = {k: list(sorted(v.items(), \
                               key=lambda x: x[1], reverse=True)) \
                       for k, v in sim.items()}
    
    # 获取接口函数
    def GetRecommendation(user):
        items = {}
        seen_items = set()
        if user in train:
            seen_items = train[user]
        for u, _ in sorted_user_sim[user][:K]:
            for item in train[u]:
                # 要去掉用户见过的
                if item not in seen_items:
                    if item not in items:
                        items[item] = 0
                    items[item] += sim[user][u]
        recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs
    
    return GetRecommendation
# 4、惩罚热门音乐的改进
def UserIIF(train, test, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似用户数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算item->user的倒排索引
    item_users = {}
    for user in train:
        for item in train[user]:
            if item not in item_users:
                item_users[item] = set()
            item_users[item].add(user)
    
    # 计算用户相似度矩阵
    sim = {}
    num = {}
    for item in item_users:
        users = item_users[item]
        for u in users:
            if u not in num:
                num[u] = 0
            num[u] += 1
            if u not in sim:
                sim[u] = {}
            for v in users:
                if u == v: continue
                if v not in sim[u]:
                    sim[u][v] = 0
                sim[u][v] += 1 / math.log(1 + len(users))
    for u in sim:
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u] * num[v])
    
    # 按照相似度排序
    sorted_user_sim = {k: list(sorted(v.items(), \
                               key=lambda x: x[1], reverse=True)) \
                       for k, v in sim.items()}
    
    # 获取接口函数
    def GetRecommendation(user):
        items = {}
        seen_items = set(train[user])
        for u, _ in sorted_user_sim[user][:K]:
            for item in train[u]:
                # 要去掉用户见过的
                if item not in seen_items:
                    if item not in items:
                        items[item] = 0
                    items[item] += sim[user][u]
        recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs
    
    return GetRecommendation
    
    
# 5. 基于物品余弦相似度的推荐
def ItemCF(train, test, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似物品数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算物品相似度矩阵
    sim = {}
    num = {}
    for user in train:
        items = train[user]
        for i in range(len(items)):
            u = items[i]
            if u not in num:
                num[u] = 0
            num[u] += 1
            if u not in sim:
                sim[u] = {}
            for j in range(len(items)):
                if j == i: continue
                v = items[j]
                if v not in sim[u]:
                    sim[u][v] = 0
                sim[u][v] += 1
    for u in sim:
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u] * num[v])
    
    # 按照相似度排序
    sorted_item_sim = {k: list(sorted(v.items(), \
                               key=lambda x: x[1], reverse=True)) \
                       for k, v in sim.items()}
    
    # 获取接口函数
    def GetRecommendation(user):
        items = {}
        seen_items = set()
        if user in train:
            seen_items = train[user]
        for item in train[user]:
            if item not in sorted_item_sim: continue
            for u, _ in sorted_item_sim[item][:K]:
                if u not in seen_items:
                    if u not in items:
                        items[u] = 0
                    items[u] += sim[item][u]
        recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs
    
    return GetRecommendation

#### 4、测评
指标包括：
- Precision
- Recall
- Coverage
- Popularity

In [4]:
class Metric():
    def __init__(self, train, test, GetRecommendation):
        ''' 
        GetRecommendation: 为某个用户获取推荐列表的接口
        '''
        self.train = train
        self.test = test
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
    
    # 为test中的每个用户进行推荐
    def getRec(self):
        recs = {}
        for user in self.test:
            # 这里主要是过滤掉协同推荐中无历史行为记录的用户
            if user not in self.train:
                continue
            rank = self.GetRecommendation(user)
            recs[user] = rank
        return recs
    
    # 精确率
    def precision(self):
        n_precision = 0
        hit = 0
        for user in self.test:
            
            # 协同过滤需要过滤掉没有历史行为的用户，其他算法可不执行
            if user not in self.train:
                continue
            
            test_items = set(self.test[user])
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1
            n_precision += len(rank)

        return round(hit / n_precision * 100, 2)
    
    #召回率
    def recall(self):
        n_recall = 0
        hit = 0
        for user in self.test:
            
            # 协同过滤需要过滤掉没有历史行为的用户
            if user not in self.train:
                continue
            
            test_items = set(self.test[user])
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1
            n_recall += len(test_items)

        return round(hit / n_recall * 100, 2)
    
    # 覆盖率
    def coverage(self):
        all_item, recom_item = set(), set()
        for user in self.test:
            
            # 协同过滤需要过滤掉没有历史行为的用户
            if user not in self.train:
                continue
            
            if user in self.train:
                for item in self.train[user]:
                    all_item.add(item)
            rank = self.recs[user]
            for item, score in rank:
                recom_item.add(item)

        return round(len(recom_item) / len(all_item) * 100, 2)
    
    # 流行度，这里用物品在训练集中出现的次数作为流行度，新颖度与流行度成反比
    def popularity(self):
        item_pop = {}
        for user in self.train:
            for item in self.train[user]:
                if item not in item_pop:
                    item_pop[item] = 0
                item_pop[item] += 1
        pop, num = 0, 0
        for user in self.test:
            
            # 协同过滤需要过滤掉没有历史行为的用户
            if user not in self.train:
                continue
            
            rank = self.recs[user]
            for item, score in rank:
                #取对数防止因长尾问题带来的被流行物品所主导的现象
                if item not in item_pop: continue
                
                pop += math.log(1 + item_pop[item])
                num += 1

        return round(pop / num, 6) # 公式应该是 测试集中的所有歌曲，在训练集中出现的次数为 ni  sum(log(ni + 1)) / |rank|
    
    #所有指标
    def eval(self):
        metric = {'Precision': self.precision(),
                  'Recall': self.recall(),
                  'Coverage': self.coverage(),
                  'Popularity': self.popularity()}
        print('Metric:', metric)
        return metric
    

#### 5、实验

In [5]:
class Experiment():
    def __init__(self, M, K, N, rt = 'UserCF'):
        ''' 
        进行M次实验
        K，TopK相似用户的个数
        N，TopN推荐物品的个数
        fp，数据路径
        rt，推荐算法类型
        '''
        self.M = M
        self.K = K
        self.N = N
        self.rt = rt
        self.alg = {'Random': Random, 'MostPopular': MostPopular,
                    'UserCF': UserCF, 'UserIIF':UserIIF,
                    'ItemCF': ItemCF}
        self.recs = {} # 这里想要定义一个函数
    
    # 单次实验
    @timmer
    def worker(self, train, test):
        getRecs = self.alg[self.rt](train, test, self.K, self.N)
        self.recs = getRecs
        metric = Metric(train, test, getRecs)
        return metric.eval()
    
    # 多次平均实验
    @timmer
    def run(self):
        import json
        metrics = {'Precision': 0, 'Recall': 0,
                   'Coverage': 0, 'Popularity': 0}
        dataset = Dataset(self.rt)
        for ii in range(self.M):
            train, test = dataset.splitData(self.M, ii)
            
            # with open('train.json', 'w') as f:
            #     json.dump(train, f, indent=4)
            # with open('test.json', 'w') as f:
            #     json.dump(test, f, indent=4)
                
            print("Experiment:{}".format(ii + 1))
            metric = self.worker(train, test)
            metrics = {k: metrics[k] + metric[k] for k in metrics}
        metrics = {k: metrics[k] / self.M for k in metrics}
        print('Average Result (M={}, K={}, N={}):{}'.format(self.M, self.K, self.N, metrics))
    
    @timmer
    def getRecs(self, user):
        return [items[0] for items in self.recs(user)]
        

In [6]:
# 1、Random算法测试
M, N = 10, 10
K = 0
random_exp = Experiment(M, K, N, rt = 'Random')
random_exp.run()

Func loadUserCFData, run time:0.6508347988128662
Func splitData, run time:9.256670713424683
Experiment:1
Metric: {'Precision': 0.07, 'Recall': 0.03, 'Coverage': 38.4, 'Popularity': 0.905073}
Func worker, run time:17.107415437698364
Func splitData, run time:9.33690881729126
Experiment:2
Metric: {'Precision': 0.04, 'Recall': 0.02, 'Coverage': 38.39, 'Popularity': 0.889715}
Func worker, run time:16.277704000473022
Func splitData, run time:9.404733657836914
Experiment:3
Metric: {'Precision': 0.03, 'Recall': 0.01, 'Coverage': 38.21, 'Popularity': 0.890806}
Func worker, run time:15.811793088912964
Func splitData, run time:9.353389024734497
Experiment:4
Metric: {'Precision': 0.01, 'Recall': 0.01, 'Coverage': 38.41, 'Popularity': 0.892845}
Func worker, run time:16.214905977249146
Func splitData, run time:9.561307907104492
Experiment:5
Metric: {'Precision': 0.01, 'Recall': 0.01, 'Coverage': 38.18, 'Popularity': 0.899176}
Func worker, run time:15.731945276260376
Func splitData, run time:9.370469

In [7]:
# 2、MostPopular算法测试
M, N = 10, 20
K = 0
random_exp = Experiment(M, K, N, rt = 'MostPopular')
random_exp.run()

Func loadUserCFData, run time:0.6481475830078125
Func splitData, run time:9.399346828460693
Experiment:1
Metric: {'Precision': 0.36, 'Recall': 0.31, 'Coverage': 0.33, 'Popularity': 3.199876}
Func worker, run time:8.704169273376465
Func splitData, run time:9.341788291931152
Experiment:2
Metric: {'Precision': 0.36, 'Recall': 0.31, 'Coverage': 0.33, 'Popularity': 3.199876}
Func worker, run time:8.621285676956177
Func splitData, run time:9.282142162322998
Experiment:3
Metric: {'Precision': 0.36, 'Recall': 0.31, 'Coverage': 0.33, 'Popularity': 3.199876}
Func worker, run time:8.534503936767578
Func splitData, run time:9.325172901153564
Experiment:4
Metric: {'Precision': 0.36, 'Recall': 0.31, 'Coverage': 0.33, 'Popularity': 3.199876}
Func worker, run time:8.76575255393982
Func splitData, run time:9.440594673156738
Experiment:5
Metric: {'Precision': 0.36, 'Recall': 0.31, 'Coverage': 0.33, 'Popularity': 3.199876}
Func worker, run time:8.54814338684082
Func splitData, run time:9.308867692947388


In [8]:
# 3、UserCF算法测试
M, N = 10, 20
for K in [5, 10, 20, 40, 80, 100]:
    random_exp = Experiment(M, K, N, rt = 'UserCF')
    random_exp.run()

Func loadUserCFData, run time:0.6595973968505859
Func splitData, run time:9.211907625198364
Experiment:1
Metric: {'Precision': 1.08, 'Recall': 0.53, 'Coverage': 24.97, 'Popularity': 1.453336}
Func worker, run time:0.44588446617126465
Func splitData, run time:9.149270057678223
Experiment:2
Metric: {'Precision': 1.09, 'Recall': 0.53, 'Coverage': 24.96, 'Popularity': 1.453789}
Func worker, run time:0.45723485946655273
Func splitData, run time:9.180076837539673
Experiment:3
Metric: {'Precision': 1.08, 'Recall': 0.53, 'Coverage': 24.95, 'Popularity': 1.453151}
Func worker, run time:0.4530761241912842
Func splitData, run time:9.136667490005493
Experiment:4
Metric: {'Precision': 1.09, 'Recall': 0.53, 'Coverage': 24.96, 'Popularity': 1.453553}
Func worker, run time:0.4518008232116699
Func splitData, run time:9.266833305358887
Experiment:5
Metric: {'Precision': 1.09, 'Recall': 0.53, 'Coverage': 24.95, 'Popularity': 1.453239}
Func worker, run time:0.44895076751708984
Func splitData, run time:9.7

In [6]:
# 4、ItemCF 算法测试
M, N = 10, 20
for K in [5, 10, 20, 40, 80, 100]:
    random_exp = Experiment(M, K, N, rt = 'ItemCF')
    random_exp.run()

Func loadUserCFData, run time:0.6899688243865967
Func splitData, run time:9.125893354415894
Experiment:1


MemoryError: 